In [1]:
"""Visualization with HoloViz."""

'Visualization with HoloViz.'

# Визуализация с HoloViz

Если вы пытались визуализировать pandas.DataFrame раньше, то вы, вероятно, сталкивались с Pandas .plot() API. Эти команды используют Matplotlib для рендеринга статических PNG или SVG в Jupyter блокнотах с использованием встроенного бэкэнда или интерактивных графиков через %matplotlib widget.

API-интерфейс Pandas .plot() стал де-факто стандартом для высокоуровневого построения графиков в Python и теперь поддерживается множеством различных библиотек, которые используют набор базовых механизмов построения графиков для обеспечения дополнительных возможностей. Библиотеки, которые в настоящее время поддерживают этот API, включают:

- [Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html) - API на основе Matplotlib, включенный в Pandas (статический или интерактивный вывод в Jupyter блокнотах).
- [xarray](https://xarray.pydata.org/en/stable/plotting.html) - API на основе Matplotlib, включенный в xarray, на основе pandas .plot API (статический или интерактивный вывод в Jupyter блокнотах).
- [hvPlot](https://hvplot.pyviz.org/) - интерактивные графики на основе HoloViews и Bokeh для данных Pandas, GeoPandas, xarray, Dask, Intake и Streamz.
- [Pandas Bokeh](https://github.com/PatrikHlobil/Pandas-Bokeh) - интерактивные графики на основе Bokeh для данных Pandas, GeoPandas и PySpark.
- [Cufflinks](https://github.com/santosjorge/cufflinks) - графические интерактивные графики для данных Pandas.
- [Plotly Express](https://plotly.com/python/pandas-backend) - интерактивные графики на основе Plotly-Express для данных Pandas; только частичная поддержка ключевых аргументов API .plot.
- [PdVega](https://altair-viz.github.io/pdvega) - интерактивные графики на основе Vega-lite в JSON-формате для данных Pandas.

В этом блокноте мы исследуем возможности стандартного API `.plot` и продемонстрируем дополнительные возможности, предоставляемые `.hvplot`, которые включают бесшовную интерактивность в развернутых информационных панелях и рендеринг на стороне сервера больших наборов данных.

Чтобы показать эти особенности, мы будем использовать набор данных в виде таблиц о землетрясениях и других запрошенных сейсмологических событиях из [Каталога землетрясений USGS](https://earthquake.usgs.gov/earthquakes/search), используя его [API](https://github.com/pyviz/holoviz/wiki/Creating-the-USGS-Earthquake-dataset). Конечно, этот набор данных является всего лишь примером; тот же подход можно использовать практически с любым табличным набором данных, и аналогичные подходы можно использовать с [наборами данных с координатной привязкой (многомерный массив)](https://hvplot.holoviz.org/user_guide/Gridded_Data.html).

Для работы с пакетом [hvplot](https://hvplot.holoviz.org/user_guide/Gridded_Data.html) понадобится настроить программное окружение (установить множество модулей).

Я предпочитаю работать с [miniconda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/download.html) и раздельными виртуальными средами.

Далее в командной строке для настройки среды окружения необходимо выполнить:

```shell
    conda create --name holoviz
    conda activate holoviz
    conda install anaconda-project
    anaconda-project download pyviz/holoviz_tutorial
    cd holoviz_tutorial
    anaconda-project run jupyter lab
```

После процесса установки всех необходимых модулей и запуска Jupyter Lab можно открыть оригинал данного блокнота: tutorial/02_Plotting.ipynb.


# Чтение данных

Здесь мы сосредоточимся на Pandas, но аналогичный подход будет работать для любого поддерживаемого типа DataFrame, включая Dask для распределенных вычислений или RAPIDS cuDF для вычислений на GPU. Этот набор данных относительно велик (2,1 млн строк), но он все равно должен уместиться в памяти на любой современной машине и, следовательно, не потребует специальных внепроцессорных или распределенных подходов, таких как Dask.

In [2]:
import hvplot.pandas  # noqa: adds hvplot method to pandas objects
import numpy as np
import pandas as pd

In [3]:
!curl -L "https://www.dropbox.com/s/m2r388lpoo7isu9/earthquakes-projected.parq" -o "earthquakes-projected.parq"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   134  100   134    0     0    312      0 --:--:-- --:--:-- --:--:--   316

100    17  100    17    0     0     19      0 --:--:-- --:--:-- --:--:--    19

100   491    0   491    0     0    318      0 --:--:--  0:00:01 --:--:--   318
100   491    0   491    0     0    318      0 --:--:--  0:00:01 --:--:--     0

  1  116M    1 1440k    0     0   602k      0  0:03:18  0:00:02  0:03:16  602k
  8  116M    8 9984k    0     0  2942k      0  0:00:40  0:00:03  0:00:37 8535k
 16  116M   16 19.6M    0     0  4590k      0  0:00:25  0:00:04  0:00:21 9360k
 24  116M   24 28.1M    0     0  5306k      0  0:00:22  0:00:05  0:00:17 9004k
 29  116M   29 34.3M    0     0  5504k      0  0

In [4]:
df = pd.read_parquet("earthquakes-projected.parq")
df.time = df.time.dt.tz_localize(None)
df = df.set_index(df.time)

ArrowMemoryError: realloc of size 16932352 failed

In [ ]:
print(df.shape)
df.head()

Чтобы сравнить подходы HoloViz с другими, мы возьмем подвыборку (1%) из большого набора данных для дальнейшей обработки любым инструментом:

In [ ]:
small_df = df.sample(frac=0.01)
print(small_df.shape)
small_df.head()

Мы будем переключаться между small_df и df в зависимости от того, работает ли метод, который мы показываем, только для небольших наборов данных, или его можно использовать для любого набора.

## Использование Pandas `.plot()`

Первое, что мы хотели бы сделать с этими данными, - это визуализировать места с землетрясениями. Итак, мы хотели бы построить диаграмму рассеяния, где x - долгота, а y - широта.

Мы можем это сделать для небольшого фрейма данных, используя API `pandas.plot` и Matplotlib:

In [ ]:
%matplotlib inline

In [ ]:
small_df.plot.scatter(x="longitude", y="latitude");

### Упражнение:

Попробуйте заменить inline на widget и посмотрите, какие интерактивные возможности доступны в Matplotlib. В некоторых случаях вам может потребоваться перезагрузить страницу и перезапустить блокнот, чтобы она отображалась правильно.

## Использование .hvplot

Как вы могли увидеть выше, Pandas API легко строит график, где вы можете посмотреть структуру краев тектонических плит, которые во многих случаях соответствуют визуальным краям континентов (например, западная сторона Африки, в центре). Вы можете создать очень похожий график с теми же аргументами, используя hvplot, после импорта `hvplot.pandas` для поддержки hvPlot в Pandas:

In [ ]:
small_df.hvplot.scatter(x="longitude", y="latitude")

Здесь, в отличие от Pandas `.plot()`, есть действие по умолчанию при наведении курсора на точки данных, чтобы показать значения местоположения, и вы всегда можете панорамировать и масштабировать, чтобы сосредоточиться на любой конкретной области интересующих данных. Масштабирование и панорамирование также работают, если вы используете бэкэнд Matplotlib `widget`.

Вы могли заметить, что многие точки в только что созданном графике лежат друг на друге. Это называется ["overplotting"](https://datashader.org/user_guide/Plotting_Pitfalls.html), и его можно избежать разными способами, например, сделав точки слегка прозрачными или объединяя данные.

### Упражнение №1

Попробуйте изменить `alpha`, установив значение 0.1 на графике выше, чтобы увидеть эффект этого подхода.

$\texttt{pythonsmall}_{d}{f.hvplot.scaer}(x = \text{'longitude'}, y = \text{'latitude'}, a = 0.1)_{d}$

In [ ]:
small_df.hvplot.scatter(x="longitude", y="latitude", alpha=0.1)

Попробуйте создать график hexbin.

$$\text{pythonsmall}_qf.\text{hvplot.hexb} \in (x = \text{'longitude'}, y = \text{'latitude'})$$

In [ ]:
small_df.hvplot.hexbin(x="longitude", y="latitude")

## Получение справки

Как можно узнать о ключевом аргументе `alpha` в первом упражнении или как вы можете узнать обо всех опциях, доступных с `hvplot`. Для этого вы можете использовать завершение табуляции в Jupyter блокноте или функцию `hvplot.help`, которые описаны в руководстве пользователя.

Для завершения табуляции вы можете нажать табуляцию после открывающей скобки в вызове `obj.hvplot.<kind>(`. Например, вы можете попробовать нажать табуляцию после частичного выражения `small_df.hvplot.scatter(<TAB>`.

Кроме того, вы можете вызвать `hvplot.help(<kind>)`, чтобы увидеть всплывающую панель документации в блокноте.

Попробуйте раскомментировать следующую строку и выполнить ее:

In [ ]:
hvplot.help("scatter")

Вы увидите, что есть много вариантов! Вы можете контролировать, какой раздел документации просматриваете, с помощью логических переключателей `generic`, `docstring` и `style`, также задокументированных в [руководстве пользователя](https://hvplot.holoviz.org/user_guide/Customization.html). Если вы запустите следующую ячейку, вы увидите, что `alpha `указана в 'Style options'.

In [ ]:
hvplot.help("scatter", style=True, generic=False)

Эти параметры стиля относятся к параметрам, которые являются частью Bokeh API. Это означает, что ключевое слово `alpha` передается непосредственно в Bokeh, как и все другие стилевые параметры. Поскольку это параметры уровня Bokeh, вы можете узнать больше, воспользовавшись функцией поиска в [документации Bokeh](https://docs.bokeh.org/en/latest/).

In [ ]:
hvplot.help("scatter", style=True, generic=False)

## Datashader

Часто приходится производить выбор еще до того, как вы понимаете свойства данных, например, выбор alpha-значения или размера ячейки для агрегирования. Такие предположения могут склонить вас к определенным аспектам данных, и, конечно же, необходимость выбросить 99% данных может скрыть закономерности, которые вы могли бы увидеть в ином случае. Для первоначального исследования нового набора данных гораздо безопаснее, если вы можете просто **просмотреть** данные, прежде чем делать какие-либо предположения о его форме или структуре, и без необходимости подвыборки.

Чтобы избежать некоторых проблем традиционных диаграмм рассеяния, мы можем использовать поддержку [Datashader](https://datashader.org/). Datashader объединяет данные в каждый пиксель без каких-либо произвольных настроек параметров, делая ваши данные видимыми немедленно, прежде чем вы узнаете, чего от них ожидать. В **hvplot** мы можем активировать эту возможность, установив **rasterize=True** для вызова Datashader перед рендерингом и **cnorm='eq_hist'** (["выравнивание гистограммы"](https://datashader.org/user_guide/Plotting_Pitfalls.html)), чтобы указать, что цветовое отображение должно адаптироваться к любому распределению данных:

In [ ]:
small_df.hvplot.scatter(x="longitude", y="latitude", rasterize=True, cnorm="eq_hist")

Мы уже можем видеть гораздо больше деталей, но помните, что мы все еще наносим на график только 1% данных (21 тыс. землетрясений). С помощью Datashader мы можем быстро и легко построить полный исходный набор данных о 2,1 млн землетрясений:

In [ ]:
df.hvplot.scatter(
    x="longitude", y="latitude", rasterize=True, cnorm="eq_hist", dynspread=True
)

Здесь вы можете увидеть все подробности из миллионов мест землетрясений. Если у вас запущен блокнот, вы можете увеличивать масштаб и видеть дополнительные детали на каждом уровне масштабирования без настройки каких-либо параметров или каких-либо предположений о форме или структуре данных.

Вы можете указать цветовое отображение **cnorm='log'** или значение по умолчанию **cnorm='linear'**, которые легче интерпретировать, но хорошей практикой является **cnorm='eq_hist'**, чтобы увидеть форму данных, прежде чем перейти к более простой для интерпретации, но потенциально скрывающей данные цветовой карте.

Вы можете узнать больше о Datashader на [datashader.org](https://datashader.org/) или на [странице Datashader на holoviews.org](https://holoviews.org/user_guide/Large_Data.html). На данный момент самое важное, что нужно знать об этом, это то, что Datashader позволяет нам удобно работать с произвольно большими наборами данных в веб-браузере.

Упражнение
Выберите подмножество данных, например только magitude >5 и нанесите их на другую цветовую карту (допустимые значения **cmap** включают 'viridis_r', 'Reds' и 'magma_r'):

$$\texttt{pythondf[df.mag>5].hvplot.scaer}(x=\text{'longitude'}, y=\text{'latitude'}, \text{datashade}=\text{True}, \text{cmap}=\text{'Reds'})$$

In [ ]:
df[df.mag > 5].hvplot.scatter(x="longitude", y="latitude", rasterize=True, cmap="Reds")

# Статистические графики

Давайте углубимся в некоторые другие возможности `.plot()` и `.hvplot()`, начиная с частоты землетрясений разной магнитуды.

| Величина | Эффект землетрясения | Расчетное количество каждый год |
|----------|----------------------|----------------------------------|
| 2,5 или менее | Обычно не ощущается, но может быть зафиксировано сейсмографом. | 900,000 |
| от 2,5 до 5,4 | Часто ощущается, но вызывает лишь незначительные повреждения. | 30,000 |
| от 5,5 до 6,0 | Незначительные повреждения зданий и других построек. | 500 |
| от 6,1 до 6,9 | Может нанести большой ущерб густонаселенным районам. | 100 |
| от 7,0 до 7,9 | Сильное землетрясение. Серьезный ущерб. | 20 |
| 8,0 или выше | Великое землетрясение. Может полностью разрушить сообщества вблизи эпицентра. Один раз в 5–10 лет | — |

В качестве первого прохода мы будем использовать гистограмму сначала с `.plot.hist`, затем с `.hvplot.hist`. Перед построением графика мы можем очистить данные, заменив любую величину меньше 0 на NaN.

In [ ]:
cleaned_df = df.copy()
cleaned_df["mag"] = df.mag.where(df.mag > 0)

In [ ]:
cleaned_df.plot.hist(y="mag", bins=50);

In [ ]:
df.hvplot.hist(y="mag", bin_range=(0, 10), bins=50)

# Упражнение
Создайте график ядерной оценки плотности (kde) величины для cleaned_df:

$$\texttt{pythonc} \leq a \neq \texttt{d}_{d}{f.hvplot.kde}(y = \text{'mag'})$$

In [ ]:
cleaned_df.hvplot.kde(y="mag")

Категориальные переменные
Далее мы классифицируем землетрясения по глубине. Вы можете прочитать обо всех переменных, доступных в этом наборе данных [здесь](https://earthquake.usgs.gov/data/comcat/data-eventterms.php). Согласно [странице USGS о глубинах землетрясений](https://earthquake.usgs.gov/data/comcat/data-eventterms.php), типичная глубина по категориям:

| Класс глубины | Глубина | 
|----------|--------------|
| мелкий | 0 - 70 км |
| средний | 70 - 300 км |
| глубокий | 300 - 700 км |

Сначала мы воспользуемся `pd.cut`, чтобы разделить `small_dataset` на категории глубины.

In [ ]:
depth_bins = [-np.inf, 70, 300, np.inf]
depth_names = ["Shallow", "Intermediate", "Deep"]
depth_class_column = pd.cut(cleaned_df["depth"], depth_bins, labels=depth_names)

In [ ]:
cleaned_df.insert(1, "depth_class", depth_class_column)

Теперь мы можем использовать новую категориальную переменную для группировки данных. Сначала мы наложим все группы на один и тот же график, используя опцию `by`:

In [ ]:
cleaned_df.hvplot.hist(y="mag", by="depth_class", alpha=0.6)

ПРИМЕЧАНИЕ: Нажмите на легенду, чтобы отключить определенные категории и посмотреть, что за ними скрывается.

Упражнение
Добавьте `subplots=True` и `width=300`, чтобы увидеть разные классы рядом, а не наложенными. Оси будут связаны, поэтому попробуйте увеличить.

In [ ]:
cleaned_df.hvplot.hist(y="mag", by="depth_class", subplots=True, width=300)

## Группировка

Что, если вам нужен один график, но вы хотите увидеть каждый класс отдельно? Вы можете использовать опцию `groupby`, чтобы получить виджет для переключения между классами, здесь, на двумерном графике (использование подмножества данных в качестве двумерных графиков может быть дорогостоящим для вычисления):

In [ ]:
cleaned_small_df = cleaned_df.sample(frac=0.01)
cleaned_small_df.hvplot.bivariate(x="mag", y="depth", groupby="depth_class")

Помимо классификации по глубине, мы можем классифицировать по величине.

| Класс магнитуды | Величина | 
|----------|--------------|
| Great | 8 or more |
| Major | 7 - 7.9 |
| Strong | 6 - 6.9 |
| Moderate | 5 - 5.9 |
| Light | 4 - 4.9 |
| Minor | 3 - 3.9 |

In [ ]:
classified_df = df[df.mag >= 3].copy()

depth_class = pd.cut(classified_df.depth, depth_bins, labels=depth_names)

classified_df["depth_class"] = depth_class

mag_bins = [2.9, 3.9, 4.9, 5.9, 6.9, 7.9, 10]
mag_names = ["Minor", "Light", "Moderate", "Strong", "Major", "Great"]
mag_class = pd.cut(classified_df.mag, mag_bins, labels=mag_names)
classified_df["mag_class"] = mag_class

categorical_df = classified_df.groupby(["mag_class", "depth_class"]).count()

Теперь, когда мы разделили данные на две категории, мы можем использовать логарифмическую тепловую карту, чтобы визуально представить эти данные как количество обнаруженных землетрясений в каждой комбинации классов глубины и магнитуды:

# Дальнейшие исследования

Как видите, hvPlot упрощает интерактивное исследование данных с помощью команд, основанных на широко используемом API Pandas `.plot ()`, но теперь поддерживает гораздо больше функций и различные типы данных. Приведенные выше визуализации касаются лишь поверхности того, что доступно на hvPlot, и вы можете изучить [веб-сайт hvPlot](https://hvplot.holoviz.org/en/docs/latest/), чтобы увидеть гораздо больше, или просто изучить его самостоятельно, используя завершение табуляции (`df.hvplot`.[TAB]).